In [1]:
# Import libraries

import sys
import pypsa
import logging
import pandas as pd
import numpy as np
import xarray as xr
from plotly import graph_objects as go
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.demand import projected_energy
from library.weather import load_weather

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}

base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 500
load_target = 15

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(discount_rate, float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'load-bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'renewables-bus', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'battery-bus', carrier='li-ion', x=midx-0.5, y=midy)
if use_h2 or biogas_limit > 0:
    network.add('Bus', 'turbine-bus', x=midx, y=midy+0.5)
if use_h2:
    network.add('Bus', 'h2-bus', carrier='h2', x=midx-0.5, y=midy+0.5)
if biogas_limit > 0:
    network.add('Bus', 'biogas-bus', x=midx, y=midy+0.9)


## Add load and backstop to load bus
network.add('Load', 'load', bus='load-bus',
            p_set=target_load
            )

network.add('Generator', 'backstop', carrier='backstop', bus='load-bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators and links to renewable bus

network.add('Generator', 'solar', carrier='solar', bus='renewables-bus',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=assumptions.loc[('solar', 'VOM'), 'value'],
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Generator', 'onwind', carrier='onwind', bus='renewables-bus',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

if use_offwind:
    network.add('Generator', 'offwind', carrier='offwind', bus='renewables-bus',
                p_nom_extendable=use_offwind,
                p_max_pu=(capacity_factor_offwind if use_offwind else [0] * len(capacity_factor_offwind)),
                p_nom_mod=assumptions.loc['offwind','unit_size'].value,
                capital_cost= annualized_capex('offwind'),
                marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
                lifetime=assumptions.loc['offwind','lifetime'].value,
                )

network.add('Link', 'Renewables load link', bus0='renewables-bus', bus1='load-bus',
            p_nom_extendable=use_offwind,
            )

## Add battery storage

network.add('Link','battery-charge', bus0='renewables-bus', bus1='battery-bus',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'battery', carrier='li-ion', bus='battery-bus',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','battery-discharge', carrier='li-ion', bus0='battery-bus', bus1='load-bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

if use_h2:
    network.add('Link', 'h2-electrolysis', carrier='h2', bus0='renewables-bus', bus1='h2-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['h2_electrolysis','unit_size'].value,
                capital_cost= annualized_capex('h2_electrolysis'),
                marginal_cost=assumptions.loc[('h2_electrolysis', 'VOM'), 'value'],
                lifetime=assumptions.loc['h2_electrolysis','lifetime'].value,
                efficiency=assumptions.loc['h2_electrolysis','efficiency'].value,
                )

    network.add('Store', 'h2', carrier='h2', bus='h2-bus',
                e_initial=(150_000 if use_h2 else 0),
                e_nom_extendable=use_h2,
                e_cyclic=True,
                capital_cost= annualized_capex('h2_storage'),
                marginal_cost=assumptions.loc['h2_storage','VOM'].value,
                lifetime=assumptions.loc['h2_storage','lifetime'].value
                )

    network.add('Link', 'H2 pipeline', carrier='h2', bus0='h2-bus', bus1='turbine-bus',
                p_nom_extendable=True,
                )

### Biogas pipeline

if biogas_limit > 0:
    network.add('Generator', 'biogas-market', carrier='biogas', bus='biogas-bus',
                p_nom_extendable=True,
                p_nom_max=biogas_limit,
                marginal_cost=assumptions.loc['biogas','cost'].value,
                lifetime=100,
                )

    network.add('Link', 'Biogas pipeline', carrier='biogas', bus0='biogas-bus', bus1='turbine-bus',
                p_nom_extendable=True,
                )

### Gas turbines
if use_h2 or biogas_limit > 0:
         
    network.add('Link', 'gas-turbine', carrier='mixedgas', bus0='turbine-bus', bus1='load-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['combined_cycle_gas_turbine','unit_size'].value,
                capital_cost= annualized_capex('combined_cycle_gas_turbine'),
                marginal_cost=assumptions.loc['combined_cycle_gas_turbine','VOM'].value,
                lifetime=assumptions.loc['combined_cycle_gas_turbine','lifetime'].value,
                efficiency=assumptions.loc['combined_cycle_gas_turbine','efficiency'].value,
                )

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add offwind constraint
if False:
    offwind_percentage = 0.5
    offwind_e = model.variables['Generator-p'].loc[:,'offwind'].sum()
    onwind_e = model.variables['Generator-p'].loc[:,'onwind'].sum()

    #offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    offwind_constraint = offwind_e - offwind_percentage * (onwind_e + offwind_e)
    model.add_constraints(offwind_constraint >= 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 633.68it/s]
INFO:linopy.io: Writing time: 0.56s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105141 duals
Objective: 2.28e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
48184 rows, 42352 cols, 127032 nonzeros  0s
38071 rows, 32239 cols, 117366 nonzeros  0s
37991 rows, 32159 cols, 117722 nonzeros  0s

Solving MIP model with:
   37991 rows
   32159 cols (0 binary, 5 integer, 0 implied int., 32154 continuous)
   117722 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.4s
 S       0       0         0   0.00%   -inf            22840527146.16     Large        0      0      0         0     8.7s
 R       0       0         0   0.00%   22707006458.27  22840175350.7      0.58%        0      0      0     39654     

('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       2245.061126                 0.0  4.923503e+05   
          biogas          500.000000                 0.0  6.551018e+05   
          offwind           0.000000                 0.0  5.450919e-14   
          onwind         6009.197143                 0.0  7.826806e+06   
          solar          6438.919620                 0.0  6.656816e+06   
Link      AC             4501.856142                 0.0  1.418579e+07   
          biogas          500.000000                 0.0  6.551018e+05   
          h2              700.000000                 0.0  2.094201e+05   
          li-ion         1914.697792                 0.0  2.523178e+06   
          mixedgas        600.000000                 0.0  4.361320e+05   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     h2             3169.348482                 0.0  1.047101e+05   
          li-ion        18453.631778                 0.0  2.596529e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  4.923503e+05  0.000000e+00         0.025035   
          biogas    0.000000e+00  6.551018e+05  0.000000e+00         0.149567   
          offwind   0.000000e+00  5.450919e-14  0.000000e+00              NaN   
          onwind    0.000000e+00  7.826806e+06  0.000000e+00         0.148684   
          solar     0.000000e+00  6.656816e+06  0.000000e+00         0.118018   
Link      AC        1.429807e+07 -1.122824e+05  1.154834e+07         0.362561   
          biogas    6.551018e+05  0.000000e+00  6.551018e+05         0.149567   
          h2        2.902628e+05 -8.084268e+04  0.000000e+00         0.047336   
          li-ion    2.630596e+06 -1.074177e+05  0.000000e+00         0.156837   
          mixedgas  7.598119e+05 -3.236799e+05  7.598119e+05         0.144561   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        1.047101e+05  2.751221e-11  0.000000e+00         0.358298   
          li-ion    2.603380e+06 -6.850995e+03  0.000000e+00         0.529801   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          offwind   0.000000e+00         0.000000e+00   
          onwind    1.232089e+07         8.100161e+09   
          solar     6.170481e+05         3.344542e+09   
Link      AC        0.000000e+00         3.918955e+08   
          biogas    0.000000e+00         0.000000e+00   
          h2        0.000000e+00         1.091117e+08   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         7.989853e+08   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         2.416216e+08   
          li-ion    0.000000e+00         3.554022e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             5.175587e+09      0.0           NaN  
          biogas               8.370499e+08      0.0           NaN  
          offwind              3.152538e-12      0.0           NaN  
          onwind               2.252522e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          biogas               0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas             4.889808e+07      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -1.921944e+05      0.0           NaN